In [3]:
# !pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 36.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 39.2 MB/s eta 0:00:00a 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125919 sha256=b3822f943afb0ae8a3dc758c403e92ad4607aa113fc210c07c2ef189f5a7085c
  Stored in directory: /tmp/pip-ephem-wheel-cache-9lp73_av/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: huggin

In [1]:
# -- fix path --
from pathlib import Path
import sys
sys.path.append(str(Path('..').resolve()))

from source.resources import *
from source.metrics import *
from source.helper import *
from source.preprocessor import *

from nltk import word_tokenize
from functools import lru_cache
import unicodedata
from collections import Counter
import nltk
import magic
from nltk.corpus import stopwords
import json 
current_dir = Path('.')


[nltk_data] Downloading package stopwords to /Users/kim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/kim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


> basic_transformer_models = ['albert-base-v1', 'albert-base-v2', 'albert-large-v1', 'albert-large-v2', 'albert-xlarge-v1', 'albert-xlarge-v2', 'albert-xxlarge-v1', 'albert-xxlarge-v2', 'bert-base-cased-finetuned-mrpc', 'bert-base-cased', 'bert-base-chinese', 'bert-base-german-cased', 'bert-base-german-dbmdz-cased', 'bert-base-german-dbmdz-uncased', 'bert-base-multilingual-cased', 'bert-base-multilingual-uncased', 'bert-base-uncased', 'bert-large-cased-whole-word-masking-finetuned-squad', 'bert-large-cased-whole-word-masking', 'bert-large-cased', 'bert-large-uncased-whole-word-masking-finetuned-squad', 'bert-large-uncased-whole-word-masking', 'bert-large-uncased', 'camembert-base', 'ctrl', 'distilbert-base-cased-distilled-squad', 'distilbert-base-cased', 'distilbert-base-german-cased', 'distilbert-base-multilingual-cased', 'distilbert-base-uncased-distilled-squad', 'distilbert-base-uncased-finetuned-sst-2-english', 'distilbert-base-uncased', 'distilgpt2', 'distilroberta-base', 'gpt2-large', 'gpt2-medium', 'gpt2-xl', 'gpt2', 'openai-gpt', 'roberta-base-openai-detector', 'roberta-base', 'roberta-large-mnli', 'roberta-large-openai-detector', 'roberta-large', 't5-11b', 't5-3b', 't5-base', 't5-large', 't5-small', 'transfo-xl-wt103', 'xlm-clm-ende-1024', 'xlm-clm-enfr-1024', 'xlm-mlm-100-1280', 'xlm-mlm-17-1280', 'xlm-mlm-en-2048', 'xlm-mlm-ende-1024', 'xlm-mlm-enfr-1024', 'xlm-mlm-enro-1024', 'xlm-mlm-tlm-xnli15-1024', 'xlm-mlm-xnli15-1024', 'xlm-roberta-base', 'xlm-roberta-large-finetuned-conll02-dutch', 'xlm-roberta-large-finetuned-conll02-spanish', 'xlm-roberta-large-finetuned-conll03-english', 'xlm-roberta-large-finetuned-conll03-german', 'xlm-roberta-large', 'xlnet-base-cased', 'xlnet-large-cased']

In [8]:
from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer('all-MiniLM-L6-v2')
# model = SentenceTransformer('all-MiniLM-L6-v2',  device=helper.get_device())
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',  device=helper.get_device())
# model = SentenceTransformer('t5-large', device=device)

        

In [11]:
# Two lists of sentences
sentences1 = ['The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome']

sentences2 = ['The dog plays in the garden',
              'A woman watches TV',
              'The new movie is so great']

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{:<35} \t\t {:>25} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

The cat sits outside                		 The dog plays in the garden 		 Score: 0.2837
A man is playing guitar             		        A woman watches TV 		 Score: -0.0327
The new movie is awesome            		 The new movie is so great 		 Score: 0.8940


In [28]:
data = load_dataset(Dataset.LexMTurk)
data = update_candidates(data)
data

,text,complex_word,complex_word_index,candidates,list_candidates
0,"in march 1992 , linux version 0.95 was the fir...",pieces,34,"[[parts], [bits], [components], [component, se...","[parts, bits, components, component, sections,..."
1,much of the water carried by these streams is ...,diverted,9,"[[redirected], [rerouted], [changed, moved], [...","[redirected, rerouted, changed, moved, drawn a..."
2,harry also becomes the worthy possessor of the...,possessor,5,"[[owner], [holder], [keeper], [buyer, master, ...","[owner, holder, keeper, buyer, master, teacher]"
3,"escapologists escape from handcuffs , straitja...",perils,24,"[[dangers], [difficulties, danger, restraints,...","[dangers, difficulties, danger, restraints, pr..."
4,"the storm continued , crossing the outer banks...",retained,13,"[[kept], [held], [maintained], [regained]]","[kept, held, maintained, regained]"
...,...,...,...,...,...
495,the marriage was marked from the outset by sex...,outset,6,"[[beginning], [start], [begining], [beggining,...","[beginning, start, begining, beggining, origin]"
496,"born as alfons karg , he was trained as a tele...",profession,19,"[[job], [career], [work, job., specialty, occu...","[job, career, work, job., specialty, occupation]"
497,"some features , however , are reminiscent of t...",reminiscent,6,"[[similar], [remindful], [reminders], [memorab...","[similar, remindful, reminders, memorable, sug..."
498,the shinto pantheon alone consists of an uncou...,consists,4,"[[is made], [is made up], [is], [contains], [i...","[is made, is made up, is, contains, is made up..."


In [17]:
row = data.iloc[0]
# print(row)
for key in row.keys():
    # print(val)
    print(key, row[key])

text in march 1992 , linux version 0.95 was the first to be capable of running x. this large version number jump was due to a feeling that a version 1.0 with no major missing pieces was imminent .
complex_word pieces
complex_word_index 34
candidates [['parts'], ['bits'], ['components'], ['component', 'sections', 'elements', 'part', 'information', 'items']]
list_candidates ['parts', 'bits', 'components', 'component', 'sections', 'elements', 'part', 'information', 'items']


In [38]:
# model = SentenceTransformer('all-MiniLM-L6-v2',  device=helper.get_device())
# model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',  device=helper.get_device())
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1',  device=helper.get_device())
def get_sentence_similarity(sent1, sent2):
    sent1_embedding = model.encode(sent1, convert_to_tensor=True)
    sent2_embedding = model.encode(sent2, convert_to_tensor=True)
    return util.cos_sim(sent1_embedding, sent2_embedding).cpu().item() * 100
sent1 = 'text in march 1992 , linux version 0.95 was the first to be capable of running x. this large version number jump was due to a feeling that a version 1.0 with no major missing pieces was imminent .'
print(get_sentence_similarity(sent1, sent1.replace('pieces', 'parts')))
print(get_sentence_similarity(sent1, sent1.replace('pieces', 'happy')))
print(get_sentence_similarity(sent1, sent1.replace('pieces', 'boats')))

tensor([[40.2312]], device='cuda:0')
99.94071125984192
tensor([[39.6727]], device='cuda:0')
96.97914123535156
tensor([[40.3001]], device='cuda:0')
98.94147515296936


In [26]:
sent1 = 'A comienzos de la década de 1980, se trasladó a Los Ángeles, en California, donde comenzó a labrarse una reputación con sus actuaciones, tanto eléctricas como acústicas.'	

sent2 = sent1.replace('labrarse', 'construirse') #	trabajar
print(get_sentence_similarity(sent1, sent2))
print(get_sentence_similarity(sent1, sent1.replace('acústicas', 'hola')))

98.65068197250366
97.53355383872986


In [42]:
# model = SentenceTransformer('all-MiniLM-L6-v2',  device=helper.get_device())
# model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',  device=helper.get_device())
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1',  device=helper.get_device())
def get_sentence_similarity(sent1, sent2):
    sent1_embedding = model.encode(sent1, convert_to_tensor=True)
    sent2_embedding = model.encode(sent2, convert_to_tensor=True)
    return util.cos_sim(sent1_embedding, sent2_embedding).cpu().item() * 100, util.dot_score(sent1_embedding, sent2_embedding).cpu().item()

In [55]:
def normalize(val, val_min, val_max):
    return safe_division((val - val_min),(val_max - val_min))
    
tokenizer = get_tokenizer('en')    
for i in range(len(data)):
    if i == 25: break
    print('='*80)
    row = data.iloc[i]
    source = row['text']
    complex_word = row['complex_word']
    tokens = tokenizer.tokenize(source)
    index = tokens.index(complex_word)
    window_size = 200
    cropped_tokens = tokens[index-window_size:index+window_size+1]
    source = ' '.join(cropped_tokens) 
    
    # print(candidates)
    candidates = row['list_candidates'] + ['cat']
    print(row['candidates'])
    source_embedding = model.encode(source, convert_to_tensor=True)
    candidates_scores = []
    for candidate in candidates: 
        target_sent = source.replace(complex_word, candidate)
        cosine_score, dot_score = get_sentence_similarity(source, target_sent)
        candidates_scores.append((candidate, cosine_score, dot_score))
    # candidates_scores = sorted(candidates_scores, key=lambda x: x[1], reverse=True)
    val_min = min(x[1] for x in candidates_scores)
    val_max = max(x[1] for x in candidates_scores)
    for candidate, cosine_score, dot_score in candidates_scores:
        cosine_score = normalize(cosine_score, val_min, val_max)
        print(f'{complex_word} {candidate:<30}\t:\t{cosine_score:.4f}\t{dot_score:.4f}')


[['parts'], ['bits'], ['components'], ['component', 'sections', 'elements', 'part', 'information', 'items']]
pieces parts                         	:	1.0000	41.5298
pieces bits                          	:	0.7087	41.3486
pieces components                    	:	0.9677	41.5027
pieces component                     	:	0.9668	41.5508
pieces sections                      	:	0.9179	41.5969
pieces elements                      	:	0.9881	41.5639
pieces part                          	:	0.9547	41.5271
pieces information                   	:	0.9788	41.5120
pieces items                         	:	0.9564	41.4192
pieces cat                           	:	0.0000	40.9242
[['redirected'], ['rerouted'], ['changed', 'moved'], ['drawn away', 'turned', 'separated', 'switched', 'split', 'altered'], ['led away', 'sent away', 'veered', 'channeled', 'deflected']]
diverted redirected                    	:	1.0000	35.3048
diverted rerouted                      	:	0.8943	34.9638
diverted changed                       	

In [1]:
import torch 
from transformers import BertTokenizer, BertForMaskedLM
from torch import cuda
from functools import lru_cache

def get_device_type():
    device = 'cuda' if cuda.is_available() else 'cpu'
    # print(device)
    device = torch.device(device)
    
@lru_cache(maxsize=1)
def load_model():
    BERT_MODEL = 'bert-large-uncased'
    print(f'Load model: {BERT_MODEL}')
    tokenizer = BertTokenizer.from_pretrained(BERT_MODEL)
    model = BertForMaskedLM.from_pretrained(BERT_MODEL)
    device = get_device_type()
    model.to(device)
    model.eval()
    
    return model, tokenizer

def generate_candidates(sent, complex_word, num_candidates=50):
    model, tokenizer = load_model()
    device = get_device_type()
    sent = sent.lower()
    masked_complex_sent = sent.replace(complex_word, "[MASK]") # replace complex word with [MASK]
    transformed_sent = f'{sent} [SEP] {masked_complex_sent}' 
    encoding = tokenizer.encode_plus(transformed_sent, add_special_tokens=True)
    # print(encoding)
    mask_token_id = tokenizer.convert_tokens_to_ids('[MASK]')
    masked_word_index = encoding['input_ids'].index(mask_token_id)
    # print(mask_token_id, masked_word_index)
    input_ids = torch.as_tensor([encoding["input_ids"]]).to(device)
    token_type_ids = torch.as_tensor([encoding["token_type_ids"]]).to(device)
    with torch.no_grad():
        outputs = model(input_ids=input_ids, token_type_ids=token_type_ids)
        predictions = outputs['logits'][0][masked_word_index]
        # print(predictions)
        predicted_ids = torch.argsort(predictions, descending=True)[:num_candidates]
        
        # print(predictions[predicted_ids]) # the score of each predicted token
        predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_ids)
        predicted_scores = predictions[predicted_ids].cpu().numpy()
        
        return predicted_tokens, predicted_scores
        # return dict(zip(predicted_tokens, predicted_scores))

In [21]:

sent = 'the cat perched on the mat.'
print(generate_candidates(sent, 'perched'))
    

(['sat', 'was', 'stood', 'perched', 'landed', 'slept', 'sits', 'jumped', 'remained', 'stayed', 'lay', 'crouched', 'fell', 'hopped', 'rested', 'spat', 'leaned', 'waited', 'climbed', 'lived', 'settled', 'died', 'sitting', 'stepped', 'sprawled', 'moved', 'sagged', 'sang', 'appeared', 'collapsed', 'flopped', 'hovered', 'stopped', 'lied', 'huddled', 'danced', 'parked', 'slumped', 'leaped', 'sleeping', 'dropped', 'knelt', 'rode', 'is', 'knocked', 'hung', 'perch', 'walked', 'floated', 'flew'], array([13.632623 , 11.902511 , 11.600557 , 11.281731 , 11.041062 ,
       10.659115 , 10.493784 , 10.257942 , 10.177241 , 10.105614 ,
       10.102157 ,  9.896737 ,  9.454384 ,  9.127811 ,  9.068984 ,
        9.048487 ,  8.820898 ,  8.809768 ,  8.790929 ,  8.770737 ,
        8.687552 ,  8.683246 ,  8.6137705,  8.449052 ,  8.390523 ,
        8.359772 ,  8.258504 ,  8.258033 ,  8.250996 ,  8.235211 ,
        8.231626 ,  8.132798 ,  8.012472 ,  8.003994 ,  7.9687157,
        7.929854 ,  7.892858 ,  7.85676

In [30]:
for i in range(len(data)):
    if i == 20: break
    print('='*80)
    row = data.iloc[i]
    source = row['text']
    complex_word = row['complex_word']
    gold_candidates = row['candidates'] #row['list_candidates']
    # print(candidates)
    source_embedding = model.encode(source, convert_to_tensor=True)
    candidates = generate_candidates(source, complex_word)[0]
    candidates = remove_word_from_list(complex_word, candidates)
    candidates_with_scores = []
    for candidate in candidates: 
        # print(source, complex_word, candidate)
        target_sent = source.replace(complex_word, candidate)
        target_sent_embedding = model.encode(target_sent, convert_to_tensor=True)
        cosine_score = util.cos_sim(source_embedding, target_sent_embedding).cpu().item()
        candidates_with_scores.append((candidate, cosine_score))
    candidates_with_scores = sorted(candidates_with_scores, key=lambda x: x[1], reverse=True)
    
    print(source)
    print(complex_word, gold_candidates)
    for candidate, score in candidates_with_scores[:10]:
        print(f'{candidate:<30}\t:\t{score:.4f}')


in march 1992 , linux version 0.95 was the first to be capable of running x. this large version number jump was due to a feeling that a version 1.0 with no major missing pieces was imminent .
pieces [['parts'], ['bits'], ['components'], ['component', 'sections', 'elements', 'part', 'information', 'items']]
parts                         	:	0.9988
piece                         	:	0.9987
elements                      	:	0.9979
points                        	:	0.9979
links                         	:	0.9977
components                    	:	0.9977
items                         	:	0.9976
bits                          	:	0.9974
material                      	:	0.9969
goods                         	:	0.9968
much of the water carried by these streams is diverted .
diverted [['redirected'], ['rerouted'], ['changed', 'moved'], ['drawn away', 'turned', 'separated', 'switched', 'split', 'altered'], ['led away', 'sent away', 'veered', 'channeled', 'deflected']]
pumped                        	:	0.9595

In [9]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

# Two lists of sentences
sentences1 = ['The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome',
             'The motive of the killing was not known.',
             'The motive of the killing was not known.',]

sentences2 = ['The dog plays in the garden',
              'A woman watches TV',
              'The new movie is so great',
              'The reason of the killing was not known.',
              'The tool of the killing was not known.',]

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

The cat sits outside 		 The dog plays in the garden 		 Score: 0.2838
A man is playing guitar 		 A woman watches TV 		 Score: -0.0327
The new movie is awesome 		 The new movie is so great 		 Score: 0.8939
The motive of the killing was not known. 		 The reason of the killing was not known. 		 Score: 0.9133
The motive of the killing was not known. 		 The tool of the killing was not known. 		 Score: 0.8389
